In [1]:
import pandas as pd
import numpy as np

import time


from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By  # 3개는 selenium이 동작하면서 웹 화면이 로딩될때 까지 기다림을 지원
from selenium.webdriver.support.ui import WebDriverWait # 웹 드라이버 미리 설치
from selenium.webdriver.support import expected_conditions as EC       

In [70]:

driver = webdriver.Chrome('C:/Users/YOONSU/chromedriver.exe')
driver.get("https://www.google.co.kr/maps/place/Hani+Lavender+Farm/data=!4m14!1m6!3m5!1s0x5fd88d61bac9f5b5:0xeb57d668b89c90b0!2sHani+Lavender+Farm!8m2!3d38.3486203!4d128.4149391!3m6!1s0x5fd88d61bac9f5b5:0xeb57d668b89c90b0!8m2!3d38.3486203!4d128.4149391!9m1!1b1?hl=en")
driver.maximize_window()
time.sleep(5)

# sort to recent review 
wait = WebDriverWait(driver, 10)
menu_bt = wait.until(EC.element_to_be_clickable(
                       (By.XPATH, '//button[@data-value=\'Sort\']'))
                   )  
menu_bt.click()

recent_rating_bt = driver.find_elements_by_xpath(
                                     '//li[@role=\'menuitemradio\']')[1]
recent_rating_bt.click()


In [ ]:
'''
# specific attraction is differ from others, so we change html code for it.
driver = webdriver.Chrome('C:/Users/YOONSU/chromedriver.exe')
driver.get("https://www.google.co.kr/maps/place/Daemyung+Delpino+Golf+%26+Resort+%26+Hotel/data=!4m14!1m2!2m1!1z64247ZS864W4IOqzqO2UhOyVpOumrOyhsO2KuCDriIjsjbDrp6TsnqU!3m10!1s0x5fd8bd426d38d6fb:0x5cd1daca122d04bf!5m2!4m1!1i2!8m2!3d38.2120109!4d128.4940943!9m1!1b1!15sCinrjbjtlLzrhbgg6rOo7ZSE7JWk66as7KGw7Yq4IOuIiOyNsOunpOyepVouIizrjbjtlLzrhbgg6rOo7ZSEIOyVpCDrpqzsobDtirgg64iI7I2w66ekIOyepZIBDHJlc29ydF9ob3RlbJoBI0NoWkRTVWhOTUc5blMwVkpRMEZuU1VOWGNrNTZYMGwzRUFF?hl=en")
driver.maximize_window()
time.sleep(5)

# sort to recent review 
wait = WebDriverWait(driver, 10)
menu_bt = wait.until(EC.element_to_be_clickable(
                       (By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(9) > button:nth-child(2) > div.IAbLGd'))
                   )  
menu_bt.click()

recent_rating_bt = driver.find_elements(By.XPATH,
                                     '//li[@role=\'menuitemradio\']')[1]
recent_rating_bt.click()
'''

In [71]:
# scroll code
# total n review (10 reviews per 1 page)
from tqdm import tqdm

# google map allows crawling until 93th scroll(maybe robots.txt disallow crawling)
page_num = 80
per = 10
pbar = tqdm(total=page_num*per)
with tqdm(total=page_num*per, position=0, leave=True) as pbar:
    for i in range(page_num):
        buffer = driver.find_elements_by_class_name('qjESne') # buffer location
        action = ActionChains(driver)

        n =len(buffer)
        action.move_to_element(buffer[n-1]).perform() # move to last buffer
        time.sleep(1)
        pbar.update(per)
    pbar.close()

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [01:59<00:00,  6.67it/s]


In [72]:
box_list = []
name_list = []
stars_list = []
review_list = []
date_list = []

boxes = driver.find_elements_by_class_name('jftiEf.fontBodyMedium') # review class


# more butten click
for box in boxes:
    try:
        more_btn = box.find_element_by_class_name('w8nwRe.kyuRq')
        more_btn.click()
        time.sleep(0.5)
    except:
        continue
# review collection
for box in boxes:

    IDs = box.find_elements_by_class_name('d4r55')

    Dates = box.find_elements_by_class_name('rsqaWe')

    Stars = box.find_elements_by_class_name('kvMYJc')

    reviews = box.find_elements_by_css_selector('span.wiI7pd')

    for ID, Date, Star, review in zip(IDs, Dates, Stars, reviews):
        name_list.append(ID.text)
        date_list.append(Date.text)
        stars_list.append(Star.get_attribute('aria-label'))
        review_list.append(review.text)


In [76]:
# review list to dataframe
review = pd.DataFrame(
    {'name': name_list,
     'rating': stars_list,
     'review': review_list,
     'date': date_list
    })
pd.set_option('display.max_rows', None)
print(len(review))
review.head()

750


,name,rating,review,date
0,Myung jin Lee,5 stars,,a day ago
1,Yong-hyun Han,5 stars,,2 days ago
2,엄계연,4 stars,(Translated by Google) The lavender season has...,a week ago
3,시골쥐,3 stars,,a week ago
4,MANO C,2 stars,"(Translated by Google) We benchmarked Furano, ...",2 weeks ago


In [77]:
eng_review = review.copy()

In [78]:
# remove korean name
eng_review['name2'] = eng_review['name'].str.findall(r'[ㄱ-ㅎㅏ-ㅣ가-힣]')
eng_review = eng_review[eng_review['name2'].str.len() == 0]

# remove korean review
eng_review['review2'] = eng_review['review'].str.findall(r'[ㄱ-ㅎㅏ-ㅣ가-힣]')
eng_review = eng_review[eng_review['review2'].str.len() == 0]

# remove name2 & review2 columns
eng_review.drop(['name2','review2'],axis=1, inplace=True)

# remove "(Translated by Google)" sentence
eng_review['review'] = eng_review['review'].str.replace('(Translated by Google) ','',regex=False)

# remove "Original" sentence
eng_review['review'] = eng_review['review'].str.replace('\(Original\)','',regex=False)

# remove not english sentence & special characters
eng_review['review'] = eng_review['review'].str.replace(r'[^A-Za-z0-9]',' ', regex=True)

# delete side blank and "\n"
eng_review['review'] = eng_review['review'].str.strip()

# replace with lowercase
eng_review['review'] = eng_review['review'].str.lower()

# remove unreviewed data
eng_review = eng_review[eng_review['review'].str.len()!=0]

print(len(eng_review))
eng_review.head()

15


,name,rating,review,date
33,Hanju Lee (Jannabi56),5 stars,good view,a month ago
85,Udit Jain,5 stars,beautiful flowers and souvenir scent shop,a month ago
112,Gabriel Palma,5 stars,ice cream closes at 4 30,10 months ago
120,John Kitcher,5 stars,really pretty june is the best time to go so ...,11 months ago
159,Shielah Mavengere,5 stars,beautiful farm well maintained and the staff ...,a year ago


In [79]:
eng_review.tail(10)

,name,rating,review,date
236,THO TRAN,5 stars,beautiful place we went there on friday but ...,a year ago
263,Tran Duong,5 stars,oki,a year ago
416,Benjawan Chinrum,4 stars,the flowers are very beautiful the place is n...,2 years ago
563,Natasha Ovie,4 stars,im verry happy because the lavendar farm good ...,2 years ago
642,Kelly Bruinders,5 stars,instagram worthy this place is very beautiful...,3 years ago
657,Evi Masfiyah,5 stars,only available on spring and summer season,3 years ago
658,Nicholas Park,5 stars,it s not as big as i thought but still purple...,3 years ago
696,alan 20,3 stars,a shop that mainly sells lavender products o...,3 years ago
729,CJ Dayrit,4 stars,great place in goseong it is important to com...,4 years ago
740,Ka Ling (Drkyling),4 stars,beautiful lavender experience but honestly a ...,4 years ago


In [80]:
eng_review.to_csv("하니라벤더.csv", encoding="utf-8")

==================================================================================